# Explore OpenReview API Responses

Notebook for examining raw API response objects from ICLR 2020-2026

In [1]:
import sys
sys.path.insert(0, '..')

import openreview
import pandas as pd
from lib.download_metadata import (
    CONFERENCE_TIMELINES,
    get_revisions_api1,
    get_revisions_api2,
    get_value_api2,
)

In [2]:
# Load API version config
api_versions = pd.read_csv('../data/api_version_detection.csv')
api_versions = api_versions[api_versions['status'] == 'Found']
api_versions

,conference,year,venue_id,api_version,domain,status
0,ICLR,2020,ICLR.cc/2020/Conference,API 1 (Legacy),NaN,Found
1,ICLR,2021,ICLR.cc/2021/Conference,API 1 (Legacy),NaN,Found
2,ICLR,2022,ICLR.cc/2022/Conference,API 1 (Legacy),NaN,Found
3,ICLR,2023,ICLR.cc/2023/Conference,API 1 (Legacy),NaN,Found
4,ICLR,2024,ICLR.cc/2024/Conference,API 2 (New),ICLR.cc/2024/Conference,Found
5,ICLR,2025,ICLR.cc/2025/Conference,API 2 (New),ICLR.cc/2025/Conference,Found
6,ICLR,2026,ICLR.cc/2026/Conference,API 2 (New),ICLR.cc/2026/Conference,Found


In [35]:
# Initialize clients
client_v1 = openreview.Client(baseurl='https://api.openreview.net', username='skonan3@gatech.edu', password='Nb7ACvzHse6WRGi')
client_v2 = openreview.api.OpenReviewClient(baseurl='https://api2.openreview.net', username='skonan3@gatech.edu', password='Nb7ACvzHse6WRGi')
print('Connected to both APIs')

Connected to both APIs


## Fetch Submissions by Year

Fetch a few submissions per year to examine response structure

In [69]:
subs = list(client_v1.get_all_notes(
    invitation=f'{api_versions.iloc[0].venue_id}/-/Blind_Submission',
    details='replies'
))

Getting V1 Notes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 2210/2213 [00:02<00:00, 751.19it/s]


In [ ]:
subs[0].id

In [70]:
vars(subs[0])['details']['replies']

[{'id': 'HylOroWRYB',
  'original': None,
  'number': 1,
  'cdate': 1571851072514,
  'ddate': None,
  'tcdate': 1571851072514,
  'tmdate': 1574665228676,
  'tddate': None,
  'forum': 'ryxz8CVYDH',
  'replyto': 'ryxz8CVYDH',
  'invitation': 'ICLR.cc/2020/Conference/Paper1130/-/Official_Review',
  'content': {'experience_assessment': 'I do not know much about this area.',
   'rating': '6: Weak Accept',
   'review_assessment:_checking_correctness_of_experiments': 'I carefully checked the experiments.',
   'review_assessment:_thoroughness_in_paper_reading': 'I read the paper thoroughly.',
   'title': 'Official Blind Review #3',
   'review': "The paper proposes a zeroth-order optimization framework that employs an RNN to modulate the sampling used to estimate gradients and a second RNN that models the parameter update. More specifically, query directions are sampled from a Gaussian distribution with a diagonal covariance whose evolution is determined by an RNN (QueryRNN). The resulting grad

In [4]:
# Store submissions by year
submissions = {}

LIMIT = 5  # Only fetch a few for exploration

for _, row in api_versions.iterrows():
    year = row['year']
    venue_id = row['venue_id']
    api_version = row['api_version']
    
    print(f"Fetching {year} ({api_version})...")
    
    if api_version == 'API 1 (Legacy)':
        invitation = f'{venue_id}/-/Blind_Submission'
        subs = client_v1.get_all_notes(
            invitation=invitation,
            details='directReplies'
        )
    else:
        subs = list(client_v2.get_all_notes(
            invitation=f'{venue_id}/-/Submission',
            details='replies'
        ))
    
    submissions[year] = subs
    print(f"  Got {len(subs)} submissions")

print(f"\nYears loaded: {list(submissions.keys())}")

Fetching 2020 (API 1 (Legacy))...


Getting V1 Notes: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 2210/2213 [00:01<00:00, 1193.94it/s]


  Got 2213 submissions
Fetching 2021 (API 1 (Legacy))...


Getting V1 Notes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 2591/2594 [00:02<00:00, 878.84it/s]


  Got 2594 submissions
Fetching 2022 (API 1 (Legacy))...


...

  Got 7404 submissions
Fetching 2025 (API 2 (New))...


Getting V2 Notes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 11660/11672 [01:02<00:00, 187.79it/s]


  Got 11672 submissions
Fetching 2026 (API 2 (New))...


Getting V2 Notes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 19585/19605 [01:30<00:00, 216.48it/s]

  Got 19605 submissions

Years loaded: [2020, 2021, 2022, 2023, 2024, 2025, 2026]


In [80]:
vars(submission).keys()

dict_keys(['id', 'original', 'number', 'cdate', 'pdate', 'odate', 'mdate', 'tcdate', 'tmdate', 'ddate', 'content', 'forum', 'referent', 'invitation', 'replyto', 'readers', 'nonreaders', 'signatures', 'writers', 'details'])

In [86]:
import datetime

In [93]:
for field in ['cdate', 'pdate', 'odate', 'mdate', 'tcdate', 'tmdate', 'ddate']:
    n = vars(submission)
    if n[field] is not None:
        print(field,datetime.datetime.fromtimestamp(n[field] / 1000).isoformat())

cdate 2019-09-25T15:21:45.770000
pdate 2019-12-19T19:15:21.248000
odate 2019-09-25T15:21:46.025000
tcdate 2019-09-25T15:21:45.770000
tmdate 2025-10-12T04:25:02.276000


In [135]:
submission.invitations

['ICLR.cc/2024/Conference/-/Submission',
 'ICLR.cc/2024/Conference/-/Post_Submission',
 'ICLR.cc/2024/Conference/Submission1647/-/Revision',
 'ICLR.cc/2024/Conference/-/Withdrawn_Submission',
 'ICLR.cc/2024/Conference/-/Edit']

In [140]:
vars(submission)

{'id': 'zzo3Sy3NSX',
 'number': 4956,
 'cdate': 1757819760402,
 'pdate': None,
 'odate': 1759896705795,
 'mdate': 1764610470222,
 'tcdate': 1757819760402,
 'tmdate': 1764610470222,
 'ddate': None,
 'content': {'title': {'value': 'Your Language Model Secretly Contains Personality Subnetworks'},
  'keywords': {'value': ['Large Language Models', 'Persona Modeling']},
  'abstract': {'value': "Large Language Models (LLMs) demonstrate remarkable flexibility in adopting different personas and behaviors. Existing approaches typically adapt such behavior through external knowledge such as prompting, retrieval-augmented generation (RAG), or fine-tuning. We ask: do LLMs really need external context or parameters to adapting to different behaviors, or do they already have such knowledge embedded to their parameters?\nIn this work, we show that LLMs already contain persona-specialized subnetworks in their parameter space. Using small calibration datasets, we identify distinct activation signatures 

In [136]:
def to_dt(s):
    return datetime.datetime.fromtimestamp(s / 1000).isoformat()

def get_value(item):
    if isinstance(item, dict):
        return item['value']
    else:
        return item
        
for year, submission_stack in submissions.items():
    print(year)
    for submission in submission_stack:
        sub_id = submission.id
        break
        print('invitation: ', get_value(submission.invitation))
        print('submitted: ', to_dt(get_value(submission.odate)))
        print('last_modified: ', to_dt(get_value(submission.tmdate)))
        print('authors: ', get_value(submission.content['authors']))
        print('title: ', get_value(submission.content['title']))
        print('abstract: ', get_value(submission.content['abstract'])[:40] + '...')
        print('keywords: ', get_value(submission.content['keywords']))
        if year in {2020, 2023}: 
            tldr_key = 'TL;DR'
        elif year == 2021:
            tldr_key = 'one-sentence_summary'
        elif year == 2022:
            tldr_key = 'one-sentence_summary'
        else:
            pass
            
        print('tldr: ', get_value(submission.content[tldr_key]))
        print('latest_pdf: ', get_value(submission.content['pdf']))
        if year == 2020:
            print('code: ', get_value(submission.content['code']))
        if year == 2021:
            print('data: ', get_value(submission.content['data']))
        if year == 2020:
            print('original_pdf', get_value(submission.content['original_pdf']))
        break

2020
2021
2022
2023
2024
2025
2026


## Examine Raw Submission Objects

In [5]:
# Pick a year to examine
YEAR = 2020
sub = submissions[YEAR][0]
print(f"Submission ID: {sub.id}")
print(f"Type: {type(sub)}")

Submission ID: ryxz8CVYDH
Type: <class 'openreview.openreview.Note'>


In [6]:
vars(sub)

{'id': 'ryxz8CVYDH',
 'original': 'HJeyo0Luwr',
 'number': 1130,
 'cdate': 1569439305770,
 'pdate': 1576800921248,
 'odate': 1569439306025,
 'mdate': None,
 'tcdate': 1569439305770,
 'tmdate': 1760257502276,
 'ddate': None,
 'content': {'title': 'Learning to Learn by Zeroth-Order Oracle',
  'authors': ['Yangjun Ruan',
   'Yuanhao Xiong',
   'Sashank Reddi',
   'Sanjiv Kumar',
   'Cho-Jui Hsieh'],
  'authorids': ['ruanyj3107@zju.edu.cn',
   'yhxiong@cs.ucla.edu',
   'sashank@google.com',
   'sanjivk@google.com',
   'chohsieh@cs.ucla.edu'],
  'keywords': ['learning to learn',
   'zeroth-order optimization',
   'black-box adversarial attack'],
  'TL;DR': 'Novel variant of learning to learn framework for zeroth-order optimization that learns both the update rule and the Gaussian sampling rule.',
  'abstract': 'In the learning to learn (L2L) framework, we cast the design of optimization algorithms as a machine learning problem and use deep neural networks to learn the update rules. In this 

In [7]:
# Examine submission attributes
print("Attributes:")
print([attr for attr in dir(sub) if not attr.startswith('_')])

Attributes:
['cdate', 'content', 'ddate', 'details', 'forum', 'from_json', 'id', 'invitation', 'mdate', 'nonreaders', 'number', 'odate', 'original', 'pdate', 'readers', 'referent', 'replyto', 'signatures', 'tcdate', 'tmdate', 'to_json', 'writers']


In [8]:
# Content keys
print("Content keys:")
print(list(sub.content.keys()))

Content keys:
['title', 'authors', 'authorids', 'keywords', 'TL;DR', 'abstract', 'pdf', 'code', 'paperhash', 'community_implementations', '_bibtex', 'original_pdf']


In [9]:
# Full content
sub.content

{'title': 'Learning to Learn by Zeroth-Order Oracle',
 'authors': ['Yangjun Ruan',
  'Yuanhao Xiong',
  'Sashank Reddi',
  'Sanjiv Kumar',
  'Cho-Jui Hsieh'],
 'authorids': ['ruanyj3107@zju.edu.cn',
  'yhxiong@cs.ucla.edu',
  'sashank@google.com',
  'sanjivk@google.com',
  'chohsieh@cs.ucla.edu'],
 'keywords': ['learning to learn',
  'zeroth-order optimization',
  'black-box adversarial attack'],
 'TL;DR': 'Novel variant of learning to learn framework for zeroth-order optimization that learns both the update rule and the Gaussian sampling rule.',
 'abstract': 'In the learning to learn (L2L) framework, we cast the design of optimization algorithms as a machine learning problem and use deep neural networks to learn the update rules. In this paper, we extend the L2L framework to zeroth-order (ZO) optimization setting, where no explicit gradient information is available. Our learned optimizer, modeled as recurrent neural network (RNN), first approximates gradient by ZO gradient estimator a

In [12]:
sub.details['directReplies']

[{'id': 'HylOroWRYB',
  'original': None,
  'number': 1,
  'cdate': 1571851072514,
  'ddate': None,
  'tcdate': 1571851072514,
  'tmdate': 1574665228676,
  'tddate': None,
  'forum': 'ryxz8CVYDH',
  'replyto': 'ryxz8CVYDH',
  'invitation': 'ICLR.cc/2020/Conference/Paper1130/-/Official_Review',
  'content': {'experience_assessment': 'I do not know much about this area.',
   'rating': '6: Weak Accept',
   'review_assessment:_checking_correctness_of_experiments': 'I carefully checked the experiments.',
   'review_assessment:_thoroughness_in_paper_reading': 'I read the paper thoroughly.',
   'title': 'Official Blind Review #3',
   'review': "The paper proposes a zeroth-order optimization framework that employs an RNN to modulate the sampling used to estimate gradients and a second RNN that models the parameter update. More specifically, query directions are sampled from a Gaussian distribution with a diagonal covariance whose evolution is determined by an RNN (QueryRNN). The resulting grad

In [14]:
# Examine replies structure (API v1: directReplies, API v2: replies)
if 'directReplies' in sub.details:
    replies = sub.details['directReplies']
else:
    replies = sub.details.get('replies', [])

print(f"Number of replies: {len(replies)}")
if replies:
    print(f"\nFirst reply keys: {list(replies[0].keys())}")

Number of replies: 4

First reply keys: ['id', 'original', 'number', 'cdate', 'ddate', 'tcdate', 'tmdate', 'tddate', 'forum', 'replyto', 'invitation', 'content', 'signatures', 'readers', 'nonreaders', 'writers']


In [15]:
# Show all reply invitations
for i, r in enumerate(replies):
    inv = r.get('invitation', r.get('invitations', ['unknown']))
    print(f"{i}: {inv}")

0: ICLR.cc/2020/Conference/Paper1130/-/Official_Review
1: ICLR.cc/2020/Conference/Paper1130/-/Official_Review
2: ICLR.cc/2020/Conference/Paper1130/-/Official_Review
3: ICLR.cc/2020/Conference/Paper1130/-/Decision


## Examine Revisions

In [17]:
sub.id

'ryxz8CVYDH'

In [37]:
from pprint import pprint
import requests

In [18]:
refs = client_v1.get_all_references(referent=sub.id, original=True)

In [ ]:
https://openreview.net/references/pdf?id=rkG6-Roor

In [39]:
response = requests.get(f'https://openreview.net/references/pdf?id={refs[43].id}')

In [36]:
pdf_binary = client_v1.get_attachment(field_name='pdf', id='rkG6-Roor')

OpenReviewException: {'name': 'ForbiddenError', 'message': 'User skonan3@gatech.edu does not have permission to see Note rkG6-Roor', 'status': 403, 'details': {'user': 'skonan3@gatech.edu', 'reqId': '2025-12-11-6833274'}}

In [38]:
refs[43].id

'rkG6-Roor'

In [32]:
refs[43].content['pdf']

'/pdf/07a4b4b413b37f2c43c5357593d6874c057cbc5b.pdf'

In [25]:
for ix, ref in enumerate(refs):
    print(ix)
    pprint(vars(ref))
    print('*'*1000)

0
{'cdate': 1569439305770,
 'content': {'_bibtex': '@inproceedings{\n'
                        'Ruan2020Learning,\n'
                        'title={Learning to Learn by Zeroth-Order Oracle},\n'
                        'author={Yangjun Ruan and Yuanhao Xiong and Sashank '
                        'Reddi and Sanjiv Kumar and Cho-Jui Hsieh},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                        'url={https://openreview.net/forum?id=ryxz8CVYDH}\n'
                        '}',
             'community_implementations': '[![CatalyzeX](/images/catalyzex_icon.svg) '
                                          '1 code '
                                          'implementation](https://www.catalyzex.com/paper/learning-to-learn-by-zeroth-order-oracle/code)',
             'original_pdf': '/attachment/3075a428235509855db7834a4ed978b6fcf08b55.pdf'},
 'ddate': None,
 'd

In [16]:
# Get revisions for a submission
YEAR = 2020
sub = submissions[YEAR][0]
venue_id = api_versions[api_versions['year'] == YEAR]['venue_id'].values[0]
api_version = api_versions[api_versions['year'] == YEAR]['api_version'].values[0]

if api_version == 'API 1 (Legacy)':
    revisions = get_revisions_api1(client_v1, sub.id, venue_id)
else:
    revisions = get_revisions_api2(client_v2, sub.id, venue_id)

print(f"Found {len(revisions)} revisions")
revisions

Found 46 revisions


[{'revision_number': 1,
  'tcdate': '2019-09-24T23:27:18.866000',
  'cdate': '2019-09-25T15:21:45.770000',
  'note_id': 'By1iRL_PH',
  'title': 'Learning to Learn by Zeroth-Order Oracle',
  'pdf': '/pdf/3075a428235509855db7834a4ed978b6fcf08b55.pdf',
  'invitation': 'ICLR.cc/2020/Conference/-/Blind_Submission'},
 {'revision_number': 2,
  'tcdate': '2019-11-15T02:15:38.078000',
  'cdate': '2019-09-25T15:21:45.770000',
  'note_id': 'HJzixCsir',
  'title': '',
  'pdf': '/pdf/3075a428235509855db7834a4ed978b6fcf08b55.pdf',
  'invitation': 'ICLR.cc/2020/Conference/-/Blind_Submission'},
 {'revision_number': 3,
  'tcdate': '2019-11-15T02:20:25.985000',
  'cdate': '2019-09-25T15:21:45.770000',
  'note_id': 'rkG6-Roor',
  'title': '',
  'pdf': '/pdf/07a4b4b413b37f2c43c5357593d6874c057cbc5b.pdf',
  'invitation': 'ICLR.cc/2020/Conference/-/Blind_Submission'},
 {'revision_number': 4,
  'tcdate': '2020-02-07T01:14:20.159000',
  'cdate': '2019-09-25T15:21:45.770000',
  'note_id': 'yes0BX_MP',
  'title

In [36]:
import requests
from bs4 import BeautifulSoup
import os

paper_id = "ryxz8CVYDH"
url = f"https://openreview.net/revisions?id={paper_id}"


# fetch the revisions page
resp = requests.get(url)
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "html.parser")

# find all links to PDFs (revision files)
pdf_links = []
for a in soup.find_all("a", href=True):
    href = a['href']
    if href.endswith(".pdf"):
        pdf_links.append(href)

print(f"Found {len(pdf_links)} PDFs")

Found 0 PDFs


In [38]:
import requests
import json
import re
import os

# === User configuration ===
EMAIL = "skonan3@gatech.edu"
PASSWORD = "Nb7ACvzHse6WRGi"
PAPER_ID = "ryxz8CVYDH"
DOWNLOAD_DIR = PAPER_ID

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# === Step 1: Log in to OpenReview ===
login_url = "https://openreview.net/login"
session = requests.Session()
login_data = {
    "email": EMAIL,
    "password": PASSWORD
}

r = session.post(login_url, data=login_data)
r.raise_for_status()

# Verify login
if "connect.sid" not in session.cookies:
    raise RuntimeError("Login failed. Check your email/password.")

print("Logged in successfully.")

# === Step 2: Fetch the revisions page ===
revisions_url = f"https://openreview.net/revisions?id={PAPER_ID}"
resp = session.get(revisions_url)
resp.raise_for_status()
html = resp.text

# === Step 3: Extract JSON from __INITIAL_STATE__ ===
m = re.search(r'window\.__INITIAL_STATE__\s*=\s*({.*});', html)
if not m:
    raise RuntimeError("Could not find __INITIAL_STATE__ JSON on page.")

data = json.loads(m.group(1))
revisions = data.get("revisions", [])

print(f"Found {len(revisions)} revisions for paper {PAPER_ID}.")

# === Step 4: Download each PDF attachment ===
for i, rev in enumerate(revisions, start=1):
    attachments = rev.get("attachments", [])
    for att in attachments:
        if att.get("mimetype") == "application/pdf":
            pdf_url = "https://openreview.net" + att["href"] if att["href"].startswith("/") else att["href"]
            filename = os.path.join(DOWNLOAD_DIR, f"v{i}_{att['filename']}")
            
            r = session.get(pdf_url)
            r.raise_for_status()
            
            with open(filename, "wb") as f:
                f.write(r.content)
            
            print(f"Saved revision {i} as {filename}")

HTTPError: 400 Client Error: Bad Request for url: https://openreview.net/login

In [29]:
r = requests.get('https://openreview.net/revisions?id=ryxz8CVYDH')

In [26]:
for n in all_notes:
    print(n.id, n.invitation)

DWWOyAyYBN ICLR.cc/2020/Conference/Paper1130/-/Decision
Byxm28RjoB ICLR.cc/2020/Conference/Paper1130/-/Official_Comment
ryePwUAsjS ICLR.cc/2020/Conference/Paper1130/-/Official_Comment
Hkxl6r0isr ICLR.cc/2020/Conference/Paper1130/-/Official_Comment
rJlWfrRoir ICLR.cc/2020/Conference/Paper1130/-/Official_Comment
HJx62E0siH ICLR.cc/2020/Conference/Paper1130/-/Official_Comment
rylVGWoAFB ICLR.cc/2020/Conference/Paper1130/-/Official_Review
BJgs4nvCFB ICLR.cc/2020/Conference/Paper1130/-/Official_Review
HylOroWRYB ICLR.cc/2020/Conference/Paper1130/-/Official_Review
ryxz8CVYDH ICLR.cc/2020/Conference/-/Blind_Submission


In [39]:
sub

Note(id = 'ryxz8CVYDH',original = 'HJeyo0Luwr',number = 1130,cdate = 1569439305770,pdate = 1576800921248,odate = 1569439306025,mdate = None,tcdate = 1569439305770,tmdate = 1760257502276,ddate = None,content = {'title': 'Learning to Learn by Zeroth-Order Oracle', 'authors': ['Yangjun Ruan', 'Yuanhao Xiong', 'Sashank Reddi', 'Sanjiv Kumar', 'Cho-Jui Hsieh'], 'authorids': ['ruanyj3107@zju.edu.cn', 'yhxiong@cs.ucla.edu', 'sashank@google.com', 'sanjivk@google.com', 'chohsieh@cs.ucla.edu'], 'keywords': ['learning to learn', 'zeroth-order optimization', 'black-box adversarial attack'], 'TL;DR': 'Novel variant of learning to learn framework for zeroth-order optimization that learns both the update rule and the Gaussian sampling rule.', 'abstract': 'In the learning to learn (L2L) framework, we cast the design of optimization algorithms as a machine learning problem and use deep neural networks to learn the update rules. In this paper, we extend the L2L framework to zeroth-order (ZO) optimizatio

In [41]:
client.get_all_references(referent='tmIiMPl4IPa', original=True)

[Note(id = 'H9eDBzyKTlg',original = 'RAIF4RUF0T',number = 6610,cdate = 1663850589709,pdate = None,odate = None,mdate = None,tcdate = 1760256575315,tmdate = 1760256575315,ddate = None,content = {'community_implementations': '[![CatalyzeX](/images/catalyzex_icon.svg) 1 code implementation](https://www.catalyzex.com/paper/factorized-fourier-neural-operators/code)', 'resubmission': '', 'student_author': ''},forum = 'tmIiMPl4IPa',referent = 'tmIiMPl4IPa',invitation = 'ICLR.cc/2023/Conference/-/Blind_Submission',replyto = None,readers = ['everyone'],nonreaders = [],signatures = ['ICLR.cc/2023/Conference'],writers = ['ICLR.cc/2023/Conference'],details = {'original': {'id': 'H9eDBzyKTlg', 'referent': 'RAIF4RUF0T', 'original': None, 'cdate': 1760256575315, 'mdate': 1760256575315, 'ddate': None, 'tcdate': 1760256575315, 'tmdate': 1760256575315, 'tddate': None, 'tauthor': 'himanshu@profillic.com', 'forum': 'RAIF4RUF0T', 'replyto': None, 'invitation': 'CatalyzeX.com/-/ICLR.cc_2023_Conference_Code'

## Compare API v1 vs v2 Structure

In [ ]:
# Compare a v1 submission (2020-2023) vs v2 (2024-2026)
sub_v1 = submissions[2020][0]
sub_v2 = submissions[2024][0]

print("=== API v1 (2020) ===")
print(f"Type: {type(sub_v1)}")
print(f"Content keys: {list(sub_v1.content.keys())[:10]}...")
print(f"Title type: {type(sub_v1.content.get('title'))}")
print(f"Title: {sub_v1.content.get('title')[:50]}...")

print("\n=== API v2 (2024) ===")
print(f"Type: {type(sub_v2)}")
print(f"Content keys: {list(sub_v2.content.keys())[:10]}...")
print(f"Title type: {type(sub_v2.content.get('title'))}")
print(f"Title: {get_value_api2(sub_v2.content.get('title'))[:50]}...")

In [ ]:
# Examine the nested value structure in API v2
print("API v2 title field (raw):")
sub_v2.content.get('title')